In [1]:
from urllib.request import Request, urlopen
import urllib.request
import datetime
import json

import pandas as pd

In [2]:
# Para obtener esta url se debe inspeccionar con F12 y entrar en la pestaña network, si esperamos 5 segundos, deberemos copiar la última fila que aparece

url = 'https://data-cloud.flightradar24.com/zones/fcgi/feed.js?faa=1&bounds=50.783%2C47.254%2C0.274%2C21.366&satellite=1&mlat=1&flarm=1&adsb=1&gnd=1&air=1&vehicles=1&estimated=1&maxage=14400&gliders=1&stats=1&enc=09ze1OJdYLT0I3QsCeHFDbe9-5E1Ltd51l4JiiQx0SU'

In [3]:
#Función para extraer los vuelos
def extractFlights(_data):
    
    result = []
    
    raw_json = _data.decode('utf8').replace("'", '"')
    temp_json = json.loads(raw_json)
    
    count = temp_json['full_count']
    stats = temp_json['stats']
    del temp_json['full_count']
    del temp_json['version']
    del temp_json['stats']
    
    lista = list(temp_json.keys())
    
    raw_flights = []
    for leg in lista:
        raw_leg = temp_json[leg]
        raw_leg.append(leg)
        raw_flights.append(raw_leg)
        
    for flight in raw_flights:
    
        template = {
            "snapID" : str(datetime.datetime.now().strftime("%Y%m%d%H%M")) + flight[19],
            "leg": flight[19],
            "id": flight[0],
            "latitude": flight[1],
            "longitude": flight[2],
            "track": flight[3],
            "altitude": flight[4],
            "speed": flight[5],
            "ot0": flight[6],
            "airSpace": flight[7],
            "acType": flight[8],
            "acPlate": flight[9],
            "snapshot": flight[10],
            "airportDep": flight[11],
            "airportArr": flight[12],
            "flightNumber": flight[13],
            "callsign": flight[16],
            "airlineOaci": flight[18],
            "airlineCode": flight[13][:2]
        }
        result.append(template)
        
    return result

In [4]:
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
data = urlopen(req, timeout=10).read()

In [5]:
raw = extractFlights(data)

In [6]:
result = pd.DataFrame(raw, columns=raw[0].keys())
result

,snapID,leg,id,latitude,longitude,track,altitude,speed,ot0,airSpace,acType,acPlate,snapshot,airportDep,airportArr,flightNumber,callsign,airlineOaci,airlineCode
0,2022101111052dccc5e6,2dccc5e6,47843F,47.919,2.206,193,45000,454,3521,T-MLAT1,LJ45,LN-AWB,1665486342,OSL,RMU,NWG720,NWG720,NWG,NW
1,2022101111052dccc26c,2dccc26c,,48.357,3.206,332,43000,450,4067,F-LFLH1,CL30,,1665486345,,,,CL30,,
2,2022101111052dcc22d2,2dcc22d2,4074E4,49.361,11.341,294,43000,455,3225,F-LKPR8,B788,G-ZBJM,1665486342,HYD,LHR,BA276,BAW276,BAW,BA
3,2022101111052dccd9a0,2dccd9a0,3CE39A,49.418,2.589,228,43000,391,1106,F-LFAQ3,PC24,D-CNMO,1665486345,HAM,FAO,,DCNMO,,
4,2022101111052dcc7bef,2dcc7bef,710120,50.363,10.508,306,42975,455,1735,F-EDVK1,B789,HZ-ARG,1665486343,JED,AMS,SV215,SVA215,SVA,SV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761,2022101111052dccd03e,2dccd03e,394A08,49.001,2.576,357,0,0,2000,F-LFPB2,B77W,F-GSQI,1665486317,CDG,DXB,AF662,AFR662,AFR,AF
762,2022101111052dccd3b5,2dccd3b5,4B5C6D,47.461,8.559,123,0,1,0000,F-LSZH1,GRND,ZEBRA4,1665486344,,,,ZEBRA4,,
763,2022101111052dccf4e7,2dccf4e7,3C674F,50.033,8.527,244,0,10,1000,F-EDDF1,A320,D-AIZO,1665486345,FRA,PRG,LH1396,DLH2LE,DLH,LH
764,2022101111052dccd1ce,2dccd1ce,3950CE,48.728,2.355,165,0,15,1000,F-LFPB2,A318,F-GUGO,1665486323,NCE,ORY,AF6213,AFR86PP,AFR,AF
